# Compute the Frost Days Climate Index as a workflow of operators

This notebook computes the **Frost Days index** defined as a workflow of Ophidia operators including:
- a set of key-value pairs as additional global attributes shared between all the tasks
- the task list as a JSON array
- some information regarding task dependencies 

Starting from the daily minimum temperature (2096-2100) TN, the Frost Days index is the number of days where $TN < 0°C$

As first step, let's connect to the Ophidia Server

In [ ]:
import sys
from PyOphidia import cube,client
cube.Cube.setclient(read_env=True)

The JSON object associated to the workflow is shown in the cell below.

The task list includes the following tasks:

1. **Import**
 - the input NetCDF data set located at ```src_path``` (set to the first workflow input parameter) is imported into the Ophidia platform, with minimum temperature in °K (see http://ophidia.cmcc.it/documentation/users/operators/OPH_IMPORTNC.html)
 - the ```measure``` is set according to the second workflow input parameter
 - data is arranged in order to operate on time series (as indicated by the ```imp_dim``` parameter)
 - the task has no dependencies
 
 
2. **Frost Days mask**
 - the *oph_apply* operator (see http://ophidia.cmcc.it/documentation/users/operators/OPH_APPLY.html) is used to identify the frost days: $\{day \mid TN(day) < 273.15\}$ 
 - we are basically creating a mask by using the *oph_predicate* primitive (see http://ophidia.cmcc.it/documentation/users/primitives/OPH_PREDICATE.html)
 - the task has a **single** dependency from the **Import** task since it exploits only one output of parent task
 

3. **Count frost days**
 - count days below the given threshold on yearly basis
 - the *oph_reduce2* operator (see http://ophidia.cmcc.it/documentation/users/operators/OPH_REDUCE2.html) is used with ```operation=sum``` and ```concept_level=y```
 - **single** dependency from **Frost Days mask**


4. **Subset**
 - *oph_subset* operator (see http://ophidia.cmcc.it/documentation/users/operators/OPH_SUBSET.html) to subset on the first year (```subset_filter=1```, ```subset_dims=time"```)
 - **single** dependency from **Count frost days**
 

5. **Export**
 - *oph_exportnc2* operator (see http://ophidia.cmcc.it/documentation/users/operators/OPH_EXPORTNC2.html) to export summer days index related to year 2096 to a NetCDF file
 - **single** dependency from **Subset**
 
 
<img src="../imgs/Frost_Days.svg" alt="Summer_Days">

In [ ]:
workflow_FirstYear = """{
        "name": "Frost Days",
        "author": "CMCC",
        "abstract": "Workflow version of the Frost Days index",
        "exec_mode": "sync",
        "ncores": "2",
        "cwd": "/",
        "tasks":
        [
                {
                        "name": "Import",
                        "operator": "oph_importnc",
                        "arguments":
                        [
                                "src_path=$1",
                                "measure=$2",
                                "import_metadata=yes",
                                "imp_dim=time",
                                "imp_concept_level=d",
                                "vocabulary=CF",
                                "hierarchy=oph_base|oph_base|oph_time",
                                "description=Min Temp"
                        ]
                },
                {
                        "name": "Frost Days mask",
                        "operator": "oph_apply",
                        "arguments":
                        [
                                "measure_type=auto",
                                "query=oph_predicate(measure,'x-273.15','<0','1','0')",
                                "description=Frost days mask"
                        ],
                        "dependencies": [
                                { "task": "Import", "type": "single" }
                        ]
                },
                {
                        "name": "Count frost days",
                        "operator": "oph_reduce2",
                        "arguments":
                        [
                                "operation=sum",
                                "dim=time",
                                "concept_level=y",
                                "description=Frost Days Count"
                        ],
                        "dependencies": [
                                { "task": "Frost Days mask", "type": "single" }
                        ]
                },
                {
                        "name": "Subset",
                        "operator": "oph_subset",
                        "arguments":
                        [
                                "subset_filter=1",
                                "subset_dims=time",
                                "description=First year"
                        ],
                        "dependencies": [
                                { "task": "Count frost days", "type": "single" }
                        ]
                },
                {
                        "name": "Export",
                        "operator": "oph_exportnc2",
                        "arguments": [
                            "output_name=Frost_days_2096",
                            "output_path=/home/ophidia/notebooks/"
                        ],
                        "dependencies": [
                            { "task": "Subset", "type": "single"}
                        ]
                }
        ]
}"""

Once the workflow is defined, it can be executed very easily on different dataset by simply specifying the proper path and variable name for the minimum temperature.

Let's define the workflow input arguments for the example

In [ ]:
path="/home/ophidia/notebooks/"
file="tasmin_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc"
variable="tasmin"

Submit the workflow

In [ ]:
cube.Cube.client.wsubmit(workflow_FirstYear, path+file, variable)

Check for Summer_days_2096.nc

In [ ]:
import glob
glob.glob('/home/ophidia/notebooks/Frost*.nc')

We can plot a map for year 2096 by considering the PID associated to the 'First year' datacube

In [ ]:
cube.Cube.list(level=2)

In [ ]:
# Get PID of 'First year'
firstyear = cube.Cube(pid='http://127.0.0.1/ophidia/.../...')

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.util import add_cyclic_point
import numpy as np
import warnings
warnings.filterwarnings("ignore")

fig = plt.figure(figsize=(15, 6), dpi=100)

#Add Geo axes to the figure with the specified projection (PlateCarree)
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)

#Draw coastline and gridlines
ax.coastlines()

gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False

data = firstyear.export_array(show_time='yes')
lat = data['dimension'][0]['values'][ : ]
lon = data['dimension'][1]['values'][ : ]
var = data['measure'][0]['values'][ : ]
var = np.reshape(var, (len(lat), len(lon)))

#Wraparound points in longitude
var_cyclic, lon_cyclic = add_cyclic_point(var, coord=np.asarray(lon))
x, y = np.meshgrid(lon_cyclic,lat)

#Define color levels for color bar
levStep = (np.nanmax(var)-np.nanmin(var))/20
clevs = np.arange(np.nanmin(var),np.nanmax(var)+levStep,levStep)

#Set filled contour plot
cnplot = ax.contourf(x, y, var_cyclic, clevs, transform=projection,cmap=plt.cm.Oranges)
plt.colorbar(cnplot,ax=ax)

ax.set_aspect('auto', adjustable=None)

plt.title('Frost Days (year 2096)')
plt.show()

Before running the other examples, empty the workspace

In [ ]:
cube.Cube.deletecontainer(container=file,force='yes')

In [ ]:
cube.Cube.list(level=2)